In [35]:
from bs4 import BeautifulSoup
from urllib import request
from pprint import pprint
from collections import defaultdict
from datetime import datetime as dt
import pandas as pd
import re
from IPython.display import clear_output

### going into archive years 

In [2]:
urlarch = 'http://www.northyorksfire.gov.uk/news-events/breaking-news/Archive'
req = request.Request(urlarch, headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36'})
content = request.urlopen(req)
soup = BeautifulSoup(content, 'html.parser')


In [3]:
urldate = []
for i in soup.find_all('a', {'title': re.compile(r'Archived news from')}):
    urladd = i.get('href')
    urldate.append(urladd)

In [4]:
urlyearlist = []
for i in urldate:
    archive_urls = 'http://www.northyorksfire.gov.uk{}/-/page/'.format(i)
    urlyearlist.append(archive_urls)

In [5]:
urlyearlist

['http://www.northyorksfire.gov.uk/news-events/breaking-news/Archive/-/year/2018/-/page/',
 'http://www.northyorksfire.gov.uk/news-events/breaking-news/Archive/-/year/2017/-/page/',
 'http://www.northyorksfire.gov.uk/news-events/breaking-news/Archive/-/year/2016/-/page/',
 'http://www.northyorksfire.gov.uk/news-events/breaking-news/Archive/-/year/2015/-/page/']

### breaking news

In [79]:

# gets the urls for the most recent 2-3 months of incident summaries

newpages = []
breakingurl = 'http://www.northyorksfire.gov.uk/news-events/breaking-news/-/page/'
page_count = 1

while True:
    try:
        urlnewpage = breakingurl + str(page_count) + '/'
        req = request.Request(urlnewpage, headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36'})
        content = request.urlopen(req)
        soup = BeautifulSoup(content, 'html.parser')
        page_count += 1
    except:
        page_count = 1
        break
    if soup.find_all('a', {'title': re.compile(r'Incident Summary')}) == []:
        page_count = 1
        break
    newpages.append(urlnewpage)
    


['http://www.northyorksfire.gov.uk/news-events/breaking-news/-/page/1/',
 'http://www.northyorksfire.gov.uk/news-events/breaking-news/-/page/2/',
 'http://www.northyorksfire.gov.uk/news-events/breaking-news/-/page/3/',
 'http://www.northyorksfire.gov.uk/news-events/breaking-news/-/page/4/',
 'http://www.northyorksfire.gov.uk/news-events/breaking-news/-/page/5/',
 'http://www.northyorksfire.gov.uk/news-events/breaking-news/-/page/6/',
 'http://www.northyorksfire.gov.uk/news-events/breaking-news/-/page/7/',
 'http://www.northyorksfire.gov.uk/news-events/breaking-news/-/page/8/',
 'http://www.northyorksfire.gov.uk/news-events/breaking-news/-/page/9/',
 'http://www.northyorksfire.gov.uk/news-events/breaking-news/-/page/10/',
 'http://www.northyorksfire.gov.uk/news-events/breaking-news/-/page/11/',
 'http://www.northyorksfire.gov.uk/news-events/breaking-news/-/page/12/']

### going through pages

In [6]:
# gets the urls for the archived incident summaries

archivepages = []

page_count = 1

for year_url in urlyearlist:
    while True:
        try:
            urlpage = year_url + str(page_count) + '/'
            req = request.Request(urlpage, headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36'})
            content = request.urlopen(req)
            soup = BeautifulSoup(content, 'html.parser')
            page_count += 1
        except:
            page_count = 1
            break
        if soup.find_all('a', {'title': re.compile(r'Incident Summary')}) == []:
            page_count = 1
            break
        archivepages.append(urlpage)
        
            


In [80]:
allpages = newpages + archivepages

### going into incident on a page 

In [94]:
# gets the urls for the incident summary pages from the breaking news aand archive pages

urlincident = []
for page in allpages:
    req = request.Request(page, headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36'})
    content = request.urlopen(req)
    soup = BeautifulSoup(content, 'html.parser')
    for i in soup.find_all('a', {'title': re.compile(r'Incident Summary')}):
        urladd = i.get('href')
        urlincident.append('http://www.northyorksfire.gov.uk{}'.format(urladd))

In [95]:
len(urlincident)

3550

In [96]:
# there are 2 of each url on the pages so this drops the duplicates

fixedurllist = list(set(urlincident))

In [97]:
len(fixedurllist)

1775

In [28]:
fixedurllist[0]

'http://www.northyorksfire.gov.uk/news-events/breaking-news/incident_summary_night_shift_friday_29th_september_2018'

In [87]:
# scrapes each paragraph from the incident summary pages

textlines = []
count = 1
for url in fixedurllist:
    clear_output(wait=True)
    
    dayreq = request.Request(url, headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36'})
    daycontent = request.urlopen(dayreq)
    daysoup = BeautifulSoup(daycontent, 'html.parser')
    print (str(count) + '/' + str(len(fixedurllist)))
    count += 1
    
    textlines = textlines + [line.text for line in daysoup.find('div', id='col2').findAll('p')]

1775/1775


In [88]:
cleantextlines = [i.replace('\xa0', '') for i in textlines if (i != '\xa0')]

In [98]:
textdf = pd.DataFrame(cleantextlines)
textdf = textdf.rename(index=str, columns={0: 'text'})

In [100]:
textdf.to_csv(r'C:\git\fire\data\Fire_Regions\NorthYorkshireFire.csv')